# 0.   Packages and Functions

In [ ]:
!pip install pandas_datareader

In [ ]:

#general
import pandas as pd
import numpy as np

#for dates and times
import random
import time
import datetime
from pandas.tseries.holiday import USFederalHolidayCalendar

#prices
import pandas_datareader as web


In [ ]:


def str_time_prop(start, end, format, prop):
    """Get a time at a proportion of a range of two formatted times.

    start and end should be strings specifying times formated in the
    given format (strftime-style), giving an interval [start, end].
    prop specifies how a proportion of the interval to be taken after
    start.  The returned time will be in the specified format.
    """

    stime = time.mktime(time.strptime(start, format))
    etime = time.mktime(time.strptime(end, format))

    ptime = stime + prop * (etime - stime)

    return time.strftime(format, time.localtime(ptime))


def random_date(start, end, prop):
    return str_time_prop(start, end, '%Y-%m-%d', prop)

In [ ]:
def weekend2weekday (transactions, col_date, col_day):


  dates2_BUY=list()
  days2_BUY=list()

  for index,row in transactions.iterrows():
    buy_date = datetime.datetime.strptime(row[col_date], "%Y-%m-%d")
    if row[col_day]=='Sat':
      buy_date2 = buy_date - datetime.timedelta(1)
      dates2_BUY.append(buy_date2)

      buy_day2='Fri'
      days2_BUY.append(buy_day2)
    elif row[col_day]=='Sun':
      buy_date = datetime.datetime.strptime(row[col_day], "%Y-%m-%d")
      buy_date2 = buy_date + datetime.timedelta(1)
      dates2_BUY.append(buy_date2)

      buy_day2='Mon'
      days2_BUY.append(buy_day2)
    else:
      buy_date2=row[col_date]
      dates2_BUY.append(buy_date2)
      buy_day2=row[col_day]
      days2_BUY.append(buy_day2)

    transactions['date_BUY_fix']=dates2_BUY
    transactions['day_BUY_fix']=days2_BUY


  return transactions

# 1. Setting investment parameters

##Temporal horizons
 In number of days
 - 1 day to 2 years

In [ ]:
horizons=[1,2,3,4,5,6,7,10,15,20,30,40,50,60,70,80,90,100,120,150,180,210,240,270,300,330,360,420,480,540,600,660,720]


##Amounts
In dollars
 - 50 to 50,000 dollars

In [ ]:
amounts=[50,100,200,300,400,500,1000,1500,2000,3000,4000,5000,10000,15000,20000,25000,50000]

#Data extraction

## Companies' stock prices

In [ ]:
#Selecting a list of companies we are interested in analyzing:
companies = ['AAPL',
            'AMZN',
            'MSFT',
            'GOOG',
            'FB',
            'VISA',
            'NVDA',
            'PG',
            'TSLA']

We are going to extract info from 10 years from the companies selected

In [ ]:
#Collecting the data of each company in a list
l_companies_prices=list()
error_companies=list()

for company in companies:
    try:
        prices= web.DataReader(company, data_source='yahoo', start='2010-10-10', end='2020-10-10')
        l_companies_prices.append(prices)
        print(company)
    
    except :
        print('There were  errors when extracting data of  ', company)
        #elimino de la lista esa empresa que no se consiguieron datos.
        error_companies.append(company)

for c in error_companies:
  companies.remove(c)
    

AAPL
AMZN
MSFT
GOOG
FB
There were  errors when extracting data of   VISA
NVDA
PG
TSLA


In [ ]:
companies

['AAPL', 'AMZN', 'MSFT', 'GOOG', 'FB', 'NVDA', 'PG', 'TSLA']

In [ ]:
len(l_companies_prices)

8

In [ ]:
len(companies)

8

In [ ]:
print(companies[0])
l_companies_prices[0].columns

AAPL


Index(['High', 'Low', 'Open', 'Close', 'Volume', 'Adj Close'], dtype='object')

In [ ]:
company_pos=companies.index("AAPL")
l_companies_prices[company_pos].loc['2010-10-11','Close']

10.548571586608887

In [ ]:
l_companies_prices[7].to_csv('PG_prices.csv')

In [ ]:
l_companies_prices[7]

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2010-10-11,4.140000,4.014000,4.088000,4.048000,856000.0,4.048000
2010-10-12,4.056000,4.006000,4.040000,4.048000,1220000.0,4.048000
2010-10-13,4.170000,4.072000,4.128000,4.108000,1591000.0,4.108000
2010-10-14,4.206000,4.080000,4.200000,4.150000,1474000.0,4.150000
2010-10-15,4.180000,4.050000,4.178000,4.108000,1423500.0,4.108000
...,...,...,...,...,...,...
2020-10-05,433.640015,419.329987,423.350006,425.679993,44722800.0,425.679993
2020-10-06,428.779999,406.049988,423.790009,413.980011,49146300.0,413.980011
2020-10-07,429.899994,413.850006,419.869995,425.299988,43127700.0,425.299988


# 3. Random transactions

In [ ]:
transactions=pd.DataFrame()
#we do 40,000 initial transactions

#id
transactions['id']=range(1,40001)

## Companies

In [ ]:
#random company
transactions["company"]=np.random.choice(companies, size=len(transactions))

## Horizons

In [ ]:
#random horizon
transactions["horizon"] = np.random.choice(horizons, size=len(transactions))


## Investment Amounts

In [ ]:
#random investment amount
transactions["amount"] = np.random.choice(amounts, size=len(transactions))


## Dates - BUY

In [ ]:
#random buy date
start='2013-10-10'
end='2018-10-10'

dates_buy=list()

for i in range(0, transactions.shape[0]):
    r_date=random_date(start,end,random.random())
    dates_buy.append(r_date)


print(len(dates_buy))
transactions['date_BUY']=dates_buy

40000


### Fixing dates (BUY)

In [ ]:
day_names=list()

for d in dates_buy:

  dt = datetime.datetime.strptime(d, '%Y-%m-%d')
  weekno=dt.weekday()

  if weekno == 0:
    dia='Mon'
    day_names.append(dia)
  elif weekno==1:
    dia='Tue'
    day_names.append(dia)
  elif weekno==2:
    dia='Wed'
    day_names.append(dia)
  elif weekno==3:
    dia='Thu'
    day_names.append(dia)
  elif weekno==4:
    dia='Fri'
    day_names.append(dia)
  elif weekno==5:
    dia='Sat'
    day_names.append(dia)
  elif weekno==6:
    dia='Sun'
    day_names.append(dia)

transactions['day_BUY']=day_names


In [ ]:
dates2_BUY=list()
days2_BUY=list()

#if date is saturday, we are going to push it to friday
#if date is sunday, we are going to push it to monday

for index,row in transactions.iterrows():
  buy_date = datetime.datetime.strptime(row['date_BUY'], "%Y-%m-%d")
  if row['day_BUY']=='Sat':
    buy_date2 = buy_date - datetime.timedelta(1)
    dates2_BUY.append(buy_date2)

    buy_day2='Fri'
    days2_BUY.append(buy_day2)
  elif row['day_BUY']=='Sun':
    buy_date = datetime.datetime.strptime(row['date_BUY'], "%Y-%m-%d")
    buy_date2 = buy_date + datetime.timedelta(1)
    dates2_BUY.append(buy_date2)

    buy_day2='Mon'
    days2_BUY.append(buy_day2)
  else:
    buy_date2=buy_date
    dates2_BUY.append(buy_date2)
    buy_day2=row['day_BUY']
    days2_BUY.append(buy_day2)


transactions['date_BUY_fix']=dates2_BUY
transactions['day_BUY_fix']=days2_BUY

# transactions=weekend2weekday(transactions,'date_BUY','day_BUY' )

In [ ]:
#checking for holidays (NYSE is closed)
cal = USFederalHolidayCalendar()
holidays = cal.holidays(start='2010-10-10', end='2020-10-10').to_pydatetime()
# if datetime.datetime(2014,01,01) in holidays:
#     print True 

l_holidays=list()
for d in dates2_BUY:
  if d in holidays:
    holiday='Holiday'
    l_holidays.append(holiday)

  else:
    holiday=''
    l_holidays.append(holiday)

transactions['holiday_BUY']=l_holidays

In [ ]:
transactions_h=transactions[transactions.holiday_BUY=='Holiday']

transactions=transactions[transactions.holiday_BUY!='Holiday']

## Prices - BUY

In [ ]:
#price when bought

l_buy_prices=list()
for index,row in transactions.iterrows():
  comp=row['company']
  buy_date=row['date_BUY_fix']
  company_pos=companies.index(comp)
  try:
    price=l_companies_prices[company_pos].loc[buy_date,'Adj Close']
  except :
    price=0

  l_buy_prices.append(price)

transactions['price_BUY']=l_buy_prices


## Dates - SELL

In [ ]:
#random sell date
dates_SELL=list()

for index,row in transactions.iterrows():

  buy_date = row['date_BUY_fix']
  sell_date = buy_date + datetime.timedelta(row['horizon'])
  
  dates_SELL.append(sell_date)

transactions['date_SELL']=dates_SELL

### Fixing dates (SELL)

In [ ]:
day_names=list()



for d in dates_SELL:

  # dt = datetime.datetime.strptime(d, '%Y-%m-%d')
  weekno=d.weekday()

  if weekno == 0:
    dia='Mon'
    day_names.append(dia)
  elif weekno==1:
    dia='Tue'
    day_names.append(dia)
  elif weekno==2:
    dia='Wed'
    day_names.append(dia)
  elif weekno==3:
    dia='Thu'
    day_names.append(dia)
  elif weekno==4:
    dia='Fri'
    day_names.append(dia)
  elif weekno==5:
    dia='Sat'
    day_names.append(dia)
  elif weekno==6:
    dia='Sun'
    day_names.append(dia)

transactions['day_SELL']=day_names


In [ ]:
dates2_SELL=list()
days2_SELL=list()

#if date is saturday, we are going to push it to friday
#if date is sunday, we are going to push it to monday

for index,row in transactions.iterrows():
  SELL_date = row['date_SELL']
  if row['day_SELL']=='Sat':
    SELL_date2 = SELL_date - datetime.timedelta(1)
    dates2_SELL.append(SELL_date2)

    SELL_day2='Fri'
    days2_SELL.append(SELL_day2)
  elif row['day_SELL']=='Sun':
    SELL_date = row['date_SELL']
    SELL_date2 = SELL_date + datetime.timedelta(1)
    dates2_SELL.append(SELL_date2)

    SELL_day2='Mon'
    days2_SELL.append(SELL_day2)
  else:
    SELL_date2=SELL_date
    dates2_SELL.append(SELL_date2)
    SELL_day2=row['day_SELL']
    days2_SELL.append(SELL_day2)


transactions['date_SELL_fix']=dates2_SELL
transactions['day_SELL_fix']=days2_SELL

# transactions=weekend2weekday(transactions,'date_BUY','day_BUY' )

In [ ]:
transactions

,id,company,horizon,amount,date_BUY,day_BUY,date_BUY_fix,day_BUY_fix,holiday_BUY,price_BUY,date_SELL,day_SELL,date_SELL_fix,day_SELL_fix
0,1,NVDA,420,500,2014-08-04,Mon,2014-08-04,Mon,,16.841738,2015-09-28,Mon,2015-09-28,Mon
1,2,PG,10,1000,2015-06-30,Tue,2015-06-30,Tue,,66.205971,2015-07-10,Fri,2015-07-10,Fri
2,3,FB,5,1500,2014-01-04,Sat,2014-01-03,Fri,,54.560001,2014-01-08,Wed,2014-01-08,Wed
3,4,PG,150,10000,2018-03-08,Thu,2018-03-08,Thu,,73.882401,2018-08-05,Sun,2018-08-06,Mon
4,5,AMZN,50,200,2014-01-17,Fri,2014-01-17,Fri,,399.609985,2014-03-08,Sat,2014-03-07,Fri
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,39996,AAPL,4,5000,2018-04-16,Mon,2018-04-16,Mon,,42.471001,2018-04-20,Fri,2018-04-20,Fri
39996,39997,AMZN,20,10000,2016-10-04,Tue,2016-10-04,Tue,,834.030029,2016-10-24,Mon,2016-10-24,Mon
39997,39998,FB,720,3000,2016-07-27,Wed,2016-07-27,Wed,,123.339996,2018-07-17,Tue,2018-07-17,Tue
39998,39999,NVDA,60,500,2015-10-27,Tue,2015-10-27,Tue,,27.721767,2015-12-26,Sat,2015-12-25,Fri


In [ ]:
#checking for holidays (NYSE is closed)
cal = USFederalHolidayCalendar()
holidays = cal.holidays(start='2010-10-10', end='2020-10-10').to_pydatetime()
# if datetime.datetime(2014,01,01) in holidays:
#     print True 

l_holidays=list()
for d in dates2_SELL:
  if d in holidays:
    holiday='Holiday'
    l_holidays.append(holiday)

  else:
    holiday=''
    l_holidays.append(holiday)

transactions['holiday_SELL']=l_holidays

## Prices - SELL

In [ ]:
#price when sold

l_sell_prices=list()
for index,row in transactions.iterrows():
  comp=row['company']
  sell_date=row['date_SELL_fix']
  company_pos=companies.index(comp)
  try:
    price=l_companies_prices[company_pos].loc[sell_date,'Adj Close']
  except :
    price=0

  l_sell_prices.append(price)

transactions['price_SELL']=l_sell_prices


In [ ]:
transactions_h=transactions[transactions.holiday_SELL=='Holiday']

transactions=transactions[transactions.holiday_SELL!='Holiday']

In [ ]:
transactions

,id,company,horizon,amount,date_BUY,day_BUY,date_BUY_fix,day_BUY_fix,holiday_BUY,price_BUY,date_SELL,day_SELL,date_SELL_fix,day_SELL_fix,holiday_SELL,price_SELL
0,1,NVDA,420,500,2014-08-04,Mon,2014-08-04,Mon,,16.841738,2015-09-28,Mon,2015-09-28,Mon,,22.711573
1,2,PG,10,1000,2015-06-30,Tue,2015-06-30,Tue,,66.205971,2015-07-10,Fri,2015-07-10,Fri,,68.499168
2,3,FB,5,1500,2014-01-04,Sat,2014-01-03,Fri,,54.560001,2014-01-08,Wed,2014-01-08,Wed,,58.230000
3,4,PG,150,10000,2018-03-08,Thu,2018-03-08,Thu,,73.882401,2018-08-05,Sun,2018-08-06,Mon,,77.642075
4,5,AMZN,50,200,2014-01-17,Fri,2014-01-17,Fri,,399.609985,2014-03-08,Sat,2014-03-07,Fri,,372.059998
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39994,39995,AMZN,2,100,2015-02-28,Sat,2015-02-27,Fri,,380.160004,2015-03-01,Sun,2015-03-02,Mon,,385.660004
39995,39996,AAPL,4,5000,2018-04-16,Mon,2018-04-16,Mon,,42.471001,2018-04-20,Fri,2018-04-20,Fri,,40.031242
39996,39997,AMZN,20,10000,2016-10-04,Tue,2016-10-04,Tue,,834.030029,2016-10-24,Mon,2016-10-24,Mon,,838.090027
39997,39998,FB,720,3000,2016-07-27,Wed,2016-07-27,Wed,,123.339996,2018-07-17,Tue,2018-07-17,Tue,,209.990005


In [ ]:
transactions_ready=transactions[['company','horizon','amount','date_BUY_fix','date_SELL_fix','price_BUY','price_SELL']]

In [ ]:
transactions_ready

,company,horizon,amount,date_BUY_fix,date_SELL_fix,price_BUY,price_SELL
0,NVDA,420,500,2014-08-04,2015-09-28,16.841738,22.711573
1,PG,10,1000,2015-06-30,2015-07-10,66.205971,68.499168
2,FB,5,1500,2014-01-03,2014-01-08,54.560001,58.230000
3,PG,150,10000,2018-03-08,2018-08-06,73.882401,77.642075
4,AMZN,50,200,2014-01-17,2014-03-07,399.609985,372.059998
...,...,...,...,...,...,...,...
39994,AMZN,2,100,2015-02-27,2015-03-02,380.160004,385.660004
39995,AAPL,4,5000,2018-04-16,2018-04-20,42.471001,40.031242
39996,AMZN,20,10000,2016-10-04,2016-10-24,834.030029,838.090027
39997,FB,720,3000,2016-07-27,2018-07-17,123.339996,209.990005


## Filtering transactions with error (Price = 0)

In [ ]:
price_error=transactions_ready[(transactions_ready.price_BUY==0) | (transactions_ready.price_SELL==0)]
print('with errors: ', price_error.shape)
print(transactions_ready.shape)
transactions_ready=pd.concat([transactions_ready, price_error, price_error]).drop_duplicates(keep=False)
print(transactions_ready.shape)

with errors:  (409, 7)
(36293, 7)
(35587, 7)


# Real Profit/Loss (Rentabilidad Real)

In [ ]:
transactions_ready['nominal_return']=(transactions_ready['price_SELL']-transactions_ready['price_BUY'])/transactions_ready['price_BUY']

In [ ]:
transactions_ready

,company,horizon,amount,date_BUY_fix,date_SELL_fix,price_BUY,price_SELL,nominal_return
0,NVDA,420,500,2014-08-04,2015-09-28,16.841738,22.711573,0.348529
1,PG,10,1000,2015-06-30,2015-07-10,66.205971,68.499168,0.034637
2,FB,5,1500,2014-01-03,2014-01-08,54.560001,58.230000,0.067265
3,PG,150,10000,2018-03-08,2018-08-06,73.882401,77.642075,0.050887
4,AMZN,50,200,2014-01-17,2014-03-07,399.609985,372.059998,-0.068942
...,...,...,...,...,...,...,...,...
39994,AMZN,2,100,2015-02-27,2015-03-02,380.160004,385.660004,0.014468
39995,AAPL,4,5000,2018-04-16,2018-04-20,42.471001,40.031242,-0.057445
39996,AMZN,20,10000,2016-10-04,2016-10-24,834.030029,838.090027,0.004868
39997,FB,720,3000,2016-07-27,2018-07-17,123.339996,209.990005,0.702530


In [ ]:
transactions_ready['investment']=''
investments=list()
for index, row in transactions_ready.iterrows():
  if row['nominal_return']>0:
    inv='GOOD'
    investments.append(inv)
  elif row['nominal_return']<0:
    inv='BAD'
    investments.append(inv)
  else:
    inv='0'
    investments.append(inv)

transactions_ready['investment']=investments

In [ ]:
transactions_ready.shape

(35587, 9)

In [ ]:
transactions_ready

,company,horizon,amount,date_BUY_fix,date_SELL_fix,price_BUY,price_SELL,nominal_return,investment
0,NVDA,420,500,2014-08-04,2015-09-28,16.841738,22.711573,0.348529,GOOD
1,PG,10,1000,2015-06-30,2015-07-10,66.205971,68.499168,0.034637,GOOD
2,FB,5,1500,2014-01-03,2014-01-08,54.560001,58.230000,0.067265,GOOD
3,PG,150,10000,2018-03-08,2018-08-06,73.882401,77.642075,0.050887,GOOD
4,AMZN,50,200,2014-01-17,2014-03-07,399.609985,372.059998,-0.068942,BAD
...,...,...,...,...,...,...,...,...,...
39994,AMZN,2,100,2015-02-27,2015-03-02,380.160004,385.660004,0.014468,GOOD
39995,AAPL,4,5000,2018-04-16,2018-04-20,42.471001,40.031242,-0.057445,BAD
39996,AMZN,20,10000,2016-10-04,2016-10-24,834.030029,838.090027,0.004868,GOOD
39997,FB,720,3000,2016-07-27,2018-07-17,123.339996,209.990005,0.702530,GOOD


In [ ]:
transactions_ready.investment.value_counts()

GOOD    24694
BAD     10525
0         368
Name: investment, dtype: int64

## Variables calculation

### Expected return

In [ ]:
l_returns=list()
l_stdDev=list()

for company in companies:
  company_pos=companies.index(company)
  prices_r=l_companies_prices[company_pos]['Adj Close']

  # Calculting log returns and std deviation
  percent_change = prices_r.pct_change()
  returns = np.log(1+percent_change)

  meanReturn_daily = returns.mean()
  stdDev_daily = returns.std()    

  meanReturn_year= meanReturn_daily * 365
  stdDev_year= stdDev_daily *stdDev_daily* 365

  l_returns.append(meanReturn_year)
  l_stdDev.append(stdDev_year)
  


In [ ]:
companies_returns=list(zip(companies,l_returns))
companies_stddev=list(zip(companies,l_stdDev))

In [ ]:
companies_returns

[('AAPL', 0.3700680379142373),
 ('AMZN', 0.4447562102652766),
 ('MSFT', 0.34874318892447237),
 ('GOOG', 0.25098799000287664),
 ('FB', 0.33424316922218367),
 ('NVDA', 0.582100040715823),
 ('PG', 0.16589283819663966),
 ('TSLA', 0.6779141428282532)]